In [2]:
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
import glob


In [33]:

data_files_path = glob.glob(r"C:\Users\storr\OneDrive - Danmarks Tekniske Universitet\Year 2\Semester 1\Deep Learning\Project\deeplearning\data_23"+'/*.csv')

In [48]:
data_da = pd.DataFrame()
data_id = pd.DataFrame()
corrupted_data = []

for data_file_path in data_files_path:

    try:
        single_day_data = pd.read_csv(data_file_path, sep=';', skiprows=1, header=None, encoding = 'latin-1')
        single_day_data = single_day_data.iloc[:-1,:-1] # filtering out last row and columns which are blank

        if data_file_path.split('_')[-2] != 'pibcic':
    
            single_day_data.columns = ['Year','Month','Day','Hour','DA PT','DA ES'] # changing column names
            
            cols = list(range(4)) + [5] #selecting only dates and Spain data columns
            

            if len(single_day_data) == 24:
                data_da = pd.concat([data_da,single_day_data.iloc[:,cols]])
            elif len(single_day_data) > 24:
                data_da = pd.concat([data_da,single_day_data.iloc[:-1,cols]])
            elif len(single_day_data) < 24:
                hour_to_add = single_day_data.iloc[-1].copy()
                hour_to_add.iloc[4:] = np.nan
                hour_to_add.iloc[3] = 24
                single_day_data_corrected = pd.concat([single_day_data,hour_to_add.to_frame().T])
                data_da = pd.concat([data_da,single_day_data_corrected.iloc[:,cols]])

        else:
            single_day_data.columns = ['Year','Month','Day','Hour','MaxES','MaxPT','MaxMO','MinES','MinPT','MinMO','AvgES','AvgPT','AvgMO']
            single_day_data = single_day_data.iloc[1:] # removing first row which are old headers not values
            cols = list(range(5)) + [7, 10] #selecting only dates and Spain data columns

            if len(single_day_data) == 24:
                data_id = pd.concat([data_id,single_day_data.iloc[:,cols]])
            elif len(single_day_data) > 24:
                data_id = pd.concat([data_id,single_day_data.iloc[:-1,cols]])
            elif len(single_day_data) < 24:
                # copying last hour data to missing hour 24
                hour_to_add = single_day_data.iloc[-1].copy()
                hour_to_add.iloc[4:] = np.nan
                hour_to_add.iloc[3] = 24
                single_day_data_corrected = pd.concat([single_day_data,hour_to_add.to_frame().T])
                data_id = pd.concat([data_id,single_day_data_corrected.iloc[:,cols]])

        if len(single_day_data) != 24:
            file = data_file_path.split("\\")[-1]
            print(f'Hours recorded in {file}:', len(single_day_data))
            corrupted_data.append(single_day_data)



    except Exception as e:
        print(e)
        print(single_day_data)
        file = data_file_path.split('\\')[-1]
        print(f'Error in file {file}')

# Intraday market dataset given with commas - changing to '.' float
data_id.iloc[:,4:] = data_id.iloc[:,4:].replace(',','.',regex=True).astype(float)


Hours recorded in marginalpdbc_20230326.1.csv: 23
Hours recorded in marginalpdbc_20231029.1.csv: 25
Hours recorded in precios_pibcic_20230326.1.csv: 23
Hours recorded in precios_pibcic_20231029.1.csv: 25


In [63]:
# Changing str columns to floats
str_columns_da = [column for column in data_da.columns if type(data_da[column].iloc[0])==str]
str_columns_id = [column for column in data_id.columns if type(data_id[column].iloc[0])==str]

for column in str_columns_da:
    data_da[column] = data_da[column].astype(float)
for column in str_columns_id:
    data_id[column] = data_id[column].astype(float)

# checking all columns are floats:
print([data_da[column].iloc[0] for column in data_da.columns])
print([data_id[column].iloc[0] for column in data_id.columns])

# saving both DA and ID data in csv files
data_da.to_csv(r"C:\Users\storr\OneDrive - Danmarks Tekniske Universitet\Year 2\Semester 1\Deep Learning\Project\deeplearning\models_santi\data_santi\data_da.csv", index=False)
data_id.to_csv(r"C:\Users\storr\OneDrive - Danmarks Tekniske Universitet\Year 2\Semester 1\Deep Learning\Project\deeplearning\models_santi\data_santi\data_id.csv", index=False)

[2023.0, 1.0, 1.0, 1.0, 0.0]
[2023.0, 1.0, 1.0, 1.0, 0.14, -4.0, -0.72]
